# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_vacation import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Create variable to share path location of data to prepare pandas to read csv file
weather_data_load = "../WeatherPy/output_files/Clean_City_Weather_Data.csv"
# weather_data_load (prints path location)

# Read csv file
clean_city_weather_data = pd.read_csv(weather_data_load)
clean_city_weather_data 



City Country  Temperature  Humidity  Cloudiness  Windspeed  \
0    jamestown      US        48.02        64         100       4.61   
1      mataura      NZ        58.48        61          61       8.63   
2      ushuaia      AR        42.46        75         100       1.14   
3      waipawa      NZ        57.36        48          52       5.91   
4     ucluelet      CA        50.09        84          97       5.95   
..         ...     ...          ...       ...         ...        ...   
553      visby      SE        37.81        93          40       5.75   
554    pangody      RU         4.39        94          86      10.11   
555  galveston      US        73.99        85           0       3.00   
556      dawei      MM        90.30        55           8       4.34   
557      sitka      US        46.29        71         100       6.91   

     Latitude  Longitude        Date  
0     42.0970   -79.2353  1650683823  
1    -46.1927   168.8643  1650684054  
2    -54.8000   -68.3000  1650684055  
3    -41.4122   175.5153  1650684055  
4     48.9329  -125.5528  1650684056  
..        ...        ...         ...  
553   57.6409    18.2960  1650684263  
554   65.8500    74.4859  1650684263  
555   29.3669   -94.9669  1650684146  
556   14.0833    98.2000  1650684264  
557   57.0531  -135.3300  1650684264  

[558 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Create a variable for locations (Latitude, Longitude)
locations_lat_lng = clean_city_weather_data[["Latitude", "Longitude"]]
# locations_lat_lng

# Create a variable for weights (Humidity)
weight_humidity = clean_city_weather_data["Humidity"].astype(float)
# weight_humidity

In [18]:
# Plot Heatmap with heat layer
fig_1 = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations=locations_lat_lng, weights=weight_humidity, 
                                 dissipating=False, max_intensity=100, point_radius=2)
fig_1.add_layer(heat_layer)
fig_1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
# Narrow down the cities with perfect weather conditions for perfect vacation:
# Temp >70 <80, Windspeed <10 mph, Cloudiness 0%
# clean_city_weather_data (# 558 rows)

vacay_weather_filter = clean_city_weather_data

# Temperature criteria
vacay_weather_filter = vacay_weather_filter.loc[(vacay_weather_filter["Temperature"] < 80) & 
                                              (vacay_weather_filter["Temperature"] > 70)]

# Windspeed criteria
vacay_weather_filter = vacay_weather_filter.loc[vacay_weather_filter["Windspeed"] < 10]

# Cloudiness criteria
vacay_weather_filter = vacay_weather_filter.loc[vacay_weather_filter["Cloudiness"] == 0]

# Drop any rows that don't meet the "perfect vacation weather"
vacay_weather_filter = vacay_weather_filter.dropna()
vacay_weather_filter


City Country  Temperature  Humidity  Cloudiness  Windspeed  \
29          coahuayana      MX        71.06        88           0       0.89   
35     arraial do cabo      BR        75.97        87           0       8.23   
82                hilo      US        78.53        78           0       9.22   
92   sao joao da barra      BR        74.84        84           0       5.70   
178              aswan      EG        70.90        16           0       3.44   
515              ginda      IN        71.20        19           0       2.13   
555          galveston      US        73.99        85           0       3.00   

     Latitude  Longitude        Date  
29    18.7333  -103.6833  1650684067  
35   -22.9661   -42.0278  1650684070  
82    19.7297  -155.0900  1650684087  
92   -21.6403   -41.0511  1650684091  
178   24.0934    32.9070  1650684121  
515   30.6945    78.4932  1650684250  
555   29.3669   -94.9669  1650684146

In [44]:
# Cleaning data and formatting: Changed date format to datetime (mm/dd/yyyy) and reset index
# vacay_weather_filter.info()

# Changed Date format
vacay_weather_filter['Date'] = pd.to_datetime(vacay_weather_filter['Date'])
vacay_weather_filter['Date'] = vacay_weather_filter['Date'].dt.strftime('%m-%d-%Y')

# Reset Index
vacay_weather_filter.reset_index(inplace=True, drop=True)

vacay_weather_filter.head()

City Country  Temperature  Humidity  Cloudiness  Windspeed  \
0         coahuayana      MX        71.06        88           0       0.89   
1    arraial do cabo      BR        75.97        87           0       8.23   
2               hilo      US        78.53        78           0       9.22   
3  sao joao da barra      BR        74.84        84           0       5.70   
4              aswan      EG        70.90        16           0       3.44   

   Latitude  Longitude        Date  
0   18.7333  -103.6833  01-01-1970  
1  -22.9661   -42.0278  01-01-1970  
2   19.7297  -155.0900  01-01-1970  
3  -21.6403   -41.0511  01-01-1970  
4   24.0934    32.9070  01-01-1970

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
hotel_df = vacay_weather_filter
hotels = []

for city in range(len(hotel_df["City"])):
    lat = hotel_df.loc[city]["Latitude"]
    lng = hotel_df.loc[city]["Longitude"]
    
    city_locations = f"{lat},{lng}"
    
    params = {"location": city_locations,
             "types": "lodging",
             "radius": 5000,
             "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()
    
    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")
        
hotel_df["Hotel"] = hotels
hotel_df
#hotel_df.info()


City                              Hotel Country  Temperature  \
0         coahuayana  El rranchito michuacan de ocampos      MX        71.06   
1    arraial do cabo                Pousada Porto Praia      BR        75.97   
2               hilo                Hilo Hawaiian Hotel      US        78.53   
3  sao joao da barra            Pousada Porto De Canoas      BR        74.84   
4              aswan                          Nuba Nile      EG        70.90   
5              ginda                        Sunil Bisht      IN        71.20   
6          galveston                     Crystal Suites      US        73.99   

   Humidity  Cloudiness  Windspeed  Latitude  Longitude        Date  \
0        88           0       0.89   18.7333  -103.6833  01-01-1970   
1        87           0       8.23  -22.9661   -42.0278  01-01-1970   
2        78           0       9.22   19.7297  -155.0900  01-01-1970   
3        84           0       5.70  -21.6403   -41.0511  01-01-1970   
4        16           0       3.44   24.0934    32.9070  01-01-1970   
5        19           0       2.13   30.6945    78.4932  01-01-1970   
6        85           0       3.00   29.3669   -94.9669  01-01-1970   

                       Nearest Hotel  
0  El rranchito michuacan de ocampos  
1                Pousada Porto Praia  
2                Hilo Hawaiian Hotel  
3            Pousada Porto De Canoas  
4                          Nuba Nile  
5                        Sunil Bisht  
6                     Crystal Suites

In [57]:
# NOTE: Do not change any of the code in this cell

hotel_locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(hotel_locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

hotel_layer = gmaps.symbol_layer(hotel_locations, fill_color="rgba(0,150,0,0.4)", stroke_color="rgba(0,0,150,0.4)", 
                                 scale=2, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig_1.add_layer(markers)
fig_1.add_layer(hotel_layer)

# Display figure
fig_1

Figure(layout=FigureLayout(height='420px'))